In [1]:
#generate spark session
from pyspark.sql import SparkSession
    
spark = (
    SparkSession
    .builder
    .appName("read from files")
    .config("spark.streaming.stopGracefullyOnShutdown",True)
    .master("local[*]")
    .getOrCreate()
    )

spark


In [2]:
from kafka import KafkaProducer
import csv
import json
import pandas as pd

# Configuration
file_path = '/home/jovyan/sparkTEST/dataset/KDDCup99.csv'  # Update the path to your CSV file
kafka_topic = 'kafka-testing'
bootstrap_servers = ['ed-kafka:29092']

def produce_csv_to_kafka(file_path, kafka_topic, bootstrap_servers):
    producer = KafkaProducer(
        bootstrap_servers=bootstrap_servers,
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )

    with open(file_path, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            producer.send(kafka_topic, value=row)
            producer.flush()

    producer.close()

produce_csv_to_kafka(file_path, kafka_topic, bootstrap_servers)


df = pd.read_csv(file_path)

# Display the CSV data using Pandas
print("Displaying the CSV data using Pandas:")
print(df.head())

# Display the schema of the DataFrame
print("\nSchema of the DataFrame:")
print(df.info())

# Optional: Display the first few rows of the DataFrame using Pandas
print("\nFirst few rows of the DataFrame:")
print(df.head())

Displaying the CSV data using Pandas:
   duration protocol_type service flag  src_bytes  dst_bytes  land  \
0         0           tcp    http   SF        181       5450     0   
1         0           tcp    http   SF        239        486     0   
2         0           tcp    http   SF        235       1337     0   
3         0           tcp    http   SF        219       1337     0   
4         0           tcp    http   SF        217       2032     0   

   wrong_fragment  urgent  hot  ...  dst_host_srv_count  \
0               0       0    0  ...                   9   
1               0       0    0  ...                  19   
2               0       0    0  ...                  29   
3               0       0    0  ...                  39   
4               0       0    0  ...                  49   

   dst_host_same_srv_rate  dst_host_diff_srv_rate  \
0                     1.0                     0.0   
1                     1.0                     0.0   
2                     1.0  